In [1]:
import numpy as np
n_features = 20
term_pred = 'short_term'
data_file_name = 'data_y10_top_20_var'
training_id_file_name = 'all_training_ID_outerloop_cohort_0_10'
endpt = 10;
eval_times = 365.25*np.r_[1, np.arange(2, endpt+1, 2)]
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'
import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import cox_nnet
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,time,event,C08DIBAG_YEAR,C02AVGDI,DPMAINT,C08OTKAG_YEAR,C11MHAGE,CL1NTRIG,C02AVGSY,...,C02POP,C08NEPAG_YEAR,C08KYSAG_YEAR,DPMEINT,MMLAD,C35BP,C09SMKYR,MMLVDS,MMLVSSFS,DPVAE
0,100016012504,3689.025,0,0,72.0,0.05,0,41,147.0,114.0,...,120.0,0,0,0.150000,3.970000,3,12,4.010000,6.030000,0.450000
1,100023004268,3689.025,0,0,64.0,0.04,0,99,89.0,94.0,...,110.0,0,0,0.130000,3.540000,2,8,3.200000,3.300000,0.600000
2,100033323702,3689.025,0,0,73.0,0.05,0,99,110.0,105.0,...,110.0,0,0,0.160000,3.810000,2,0,3.600000,4.000000,0.490000
3,100048415270,1169.000,0,0,68.0,0.06,0,99,49.0,93.0,...,90.0,18,0,0.140000,3.320000,2,13,3.180000,3.960000,0.450000
4,100056526386,3689.025,0,0,76.0,0.07,0,99,91.0,118.0,...,116.0,0,0,0.180000,4.390000,2,0,2.800000,3.270000,0.590000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,3689.025,0,0,63.0,0.03,0,58,84.0,98.0,...,100.0,0,0,0.150000,3.430000,1,0,3.410000,3.730000,0.370000
4310,416771521620,3689.025,0,0,82.0,0.04,0,99,38.0,111.0,...,110.0,0,0,0.100000,3.130000,2,0,3.110000,3.200000,0.830000
4311,416783315386,3689.025,0,0,64.0,0.04,0,99,47.0,109.0,...,100.0,0,0,0.127864,3.415775,2,3,3.162774,3.230787,0.588047
4312,416796224310,3689.025,0,0,69.0,0.04,0,99,40.0,108.0,...,100.0,0,0,0.080000,4.640000,2,12,3.260000,4.670000,0.790000


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,10,11,12,13,14,15,16,17,18,19
0,100016012504,0,3689.025,-0.092185,0.270520,0.157052,-0.089885,-5.481100,0.928896,0.535854,...,1.421847,-0.061243,-0.090552,0.748108,0.953969,1.587715,1.311295,1.878146,1.230919,-0.939192
1,100023004268,0,3689.025,-0.092185,-0.517462,-0.528700,-0.089885,0.217005,0.116031,-1.190926,...,0.415520,-0.061243,-0.090552,0.087959,0.024043,0.131635,0.645794,0.025367,-0.072784,0.045389
2,100033323702,0,3689.025,-0.092185,0.369017,0.157052,-0.089885,0.217005,0.410344,-0.241197,...,0.415520,-0.061243,-0.090552,1.078183,0.607950,0.131635,-0.685209,0.940320,0.261499,-0.676637
3,100048415270,0,1169.000,-0.092185,-0.123471,0.842804,-0.089885,0.217005,-0.444566,-1.277265,...,-1.597133,16.114420,-0.090552,0.418034,-0.451733,0.131635,1.477670,-0.020380,0.242397,-0.939192
4,100056526386,0,3689.025,-0.092185,0.664511,1.528556,-0.089885,0.217005,0.144060,0.881210,...,1.019316,-0.061243,-0.090552,1.738332,1.862269,0.131635,-0.685209,-0.889585,-0.087111,-0.020249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,416761219907,0,3689.025,-0.092185,-0.615960,-1.214452,-0.089885,-3.810966,0.045956,-0.845570,...,-0.590806,-0.061243,-0.090552,0.748108,-0.213845,-1.324446,-0.685209,0.505717,0.132561,-1.464301
4310,416771521620,0,3689.025,-0.092185,1.255497,-0.528700,-0.089885,0.217005,-0.598730,0.276837,...,0.415520,-0.061243,-0.090552,-0.902264,-0.862631,0.131635,-0.685209,-0.180497,-0.120539,1.555080
4311,416783315386,0,3689.025,-0.092185,-0.517462,-0.528700,-0.089885,0.217005,-0.472596,0.104159,...,-0.590806,-0.061243,-0.090552,0.017451,-0.244607,0.131635,-0.186083,-0.059783,-0.105837,-0.033068
4312,416796224310,0,3689.025,-0.092185,-0.024974,-0.528700,-0.089885,0.217005,-0.570700,0.017820,...,-0.590806,-0.061243,-0.090552,-1.562413,2.402923,0.131635,1.311295,0.162610,0.581455,1.292525


In [9]:
# Outerloop:
for fold in range(25):

    #fold = 24

    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    model_params = dict(node_map = None, input_split = None)
    search_params = dict(method = "nesterov"
                         , learning_rate=0.0001
                         , momentum=0.9
                         , max_iter=200000
                         , stop_threshold=0.995
                         , patience=1000
                         , patience_incr=2
                         , rand_seed = 9114
                         , eval_step=23
                         , lr_decay = 0.9
                         , lr_growth = 1.0)

    # #train final model
    L2_reg = 0.1
    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values
    model_params = dict(node_map = None, input_split = None, L2_reg=0.1)
    cox_nnet_model, cox_nnet_cost_iter = cox_nnet.trainCoxMlp(featurespace_train
                                                              , train_df.time.values
                                                              , train_df.event.values
                                                              , model_params
                                                              , search_params
                                                              , verbose=True)
    cox_nnet_theta_train = cox_nnet_model.predictNewData(featurespace_train)
    cox_nnet_theta_test = cox_nnet_model.predictNewData(featurespace_test)

    #discrimination on train, test sets
    c_train = concordance_index(train_df.time,-cox_nnet_theta_train, train_df.event)
    print('C on training set:' + str(c_train))
    c_test = concordance_index(test_df.time,-cox_nnet_theta_test, test_df.event)
    print('C on test set:' + str(c_test))

    predicted_risk = cox_nnet_theta_test
    predicted_risk = pd.DataFrame(data=predicted_risk)
    times, H0 = cox_basehaz(cox_nnet_theta_train.astype('float32').flatten(), train_df.time.values, train_df.event.values.astype(bool))
    trained_time = pd.DataFrame(data = times)
    pred_prob_surv = cox_pred_surv(cox_nnet_theta_test.astype('float32').flatten(), H0)
    pred_prob_surv = pd.DataFrame(data=pred_prob_surv)


    savedir = os.path.join(work_dir,'csv_files/cox_nnet/'+str(n_features)+'_features/'+term_pred)
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1

    predicted_risk.to_csv(savedir+'/lp_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    pred_prob_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)
    trained_time.to_csv(savedir+'/trained_time_fold_'+str(actual_fold)+'.csv', index = None, header = True)




FOLD 0:

Using nesterov accelerated gradient
training model
cost: 0.904990, iteration: 0
cost: 0.899586, iteration: 23
cost: 0.892231, iteration: 46
cost: 0.884767, iteration: 69
cost: 0.877356, iteration: 92
cost: 0.870012, iteration: 115
cost: 0.862734, iteration: 138
cost: 0.855524, iteration: 161
cost: 0.848379, iteration: 184
cost: 0.841301, iteration: 207
cost: 0.834287, iteration: 230
cost: 0.827337, iteration: 253
cost: 0.820451, iteration: 276
cost: 0.813629, iteration: 299
cost: 0.806869, iteration: 322
cost: 0.800170, iteration: 345
cost: 0.793534, iteration: 368
cost: 0.786958, iteration: 391
cost: 0.780443, iteration: 414
cost: 0.773987, iteration: 437
cost: 0.767591, iteration: 460
cost: 0.761253, iteration: 483
cost: 0.754974, iteration: 506
cost: 0.748752, iteration: 529
cost: 0.742587, iteration: 552
cost: 0.736478, iteration: 575
cost: 0.730426, iteration: 598
cost: 0.724429, iteration: 621
cost: 0.718488, iteration: 644
cost: 0.712600, iteration: 667
cost: 0.706767,